# Deploy

Deploying a model, following https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-deploy-model?view=azureml-api-2

**NOTE**: DON'T RUN THIS YET! 

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="1b1ae7cf-df24-428b-8bb9-e4dd07869ac9",
    resource_group_name="SummerProjects2023",
    workspace_name="Nanostics_ML_Workspace",
)

In [3]:
registered_model_name = "wisconsin-bca"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

1


## Creating an Endpoint

This is where we can communicate with our model

In [5]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "wisconsin-bca-endpoint-" + str(uuid.uuid4())[:8]

In [6]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="an endpoint for our Wisconsin BCa model",
    auth_mode="key",
    tags={
        "training_dataset": "wisconsin-bca",
    },
)

## Create Endpoint

In [7]:
# create the online endpoint
# expect the endpoint to take approximately 2 minutes.

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

## Deploying!

Deploying our model (wisconsin-bca) to the endpoint above

In [10]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of our registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

In [11]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

# blue deployment takes 100% traffic
# expect the deployment to take approximately 8 to 10 minutes.
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint wisconsin-bca-endpoint-c1f57e99 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored
Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


...........................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://wisconsin-bca-endpoint-c1f57e99.canadacentral.inference.ml.azure.com/score', 'openapi_uri': 'https://wisconsin-bca-endpoint-c1f57e99.canadacentral.inference.ml.azure.com/swagger.json', 'name': 'wisconsin-bca-endpoint-c1f57e99', 'description': 'an endpoint for our Wisconsin BCa model', 'tags': {'training_dataset': 'wisconsin-bca'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/1b1ae7cf-df24-428b-8bb9-e4dd07869ac9/resourcegroups/summerprojects2023/providers/microsoft.machinelearningservices/workspaces/nanostics_ml_workspace/onlineendpoints/wisconsin-bca-endpoint-c1f57e99', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1b1ae7cf-df24-428b-8bb9-e4dd07869ac9/providers/Microsoft.MachineLearningServices/locations/canadacentral/mfeOperationsStatus/oe:d5bae791-0457-4d17-9461-6abc1fa3803c:ce4d7d12-41d5-47a9-9ec3-f4be26588c65?api-version=2022-02

In [12]:
# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

Name: wisconsin-bca-endpoint-c1f57e99
Status: Succeeded
Description: an endpoint for our Wisconsin BCa model
{'blue': 100}
https://wisconsin-bca-endpoint-c1f57e99.canadacentral.inference.ml.azure.com/score


### Testing the endpoint

In [3]:
# test the blue deployment with the sample data
ml_client.online_endpoints.invoke(
    endpoint_name='wisconsin-bca-endpoint-c1f57e99',
    deployment_name="blue",
    request_file="./deploy/sample-request.json"
)


'[1]'